Version 1:

Do the airflow tutorial https://airflow.apache.org/tutorial.html and then set up the following:

I would like you to build a DAG (airflow term) that runs a Postgres Query (join and group by) and inserts the result into the database again. Setting a Postgres database can be done on your local machine (google is your friend :-) )

Hint 1: To connect airflow to the database, you should the airflow conn_id with a saved connection in your airflow web interface. Hint 2: You can do all this with the PostgresOperator from airflow.

In [5]:
import airflow

# The DAG object; we'll need this to instantiate a DAG
from airflow import DAG

# Operators; we need this to operate!
from airflow.operators.bash_operator import BashOperator
from airflow.operators.postgres_operator import PostgresOperator
from airflow.operators.python_operator import PythonOperator

from datetime import datetime, timedelta

In [11]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': airflow.utils.dates.days_ago(1),
    'email': ['tjdcevans@gmail.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
}

dag = DAG('asana_postgres', default_args=default_args, schedule_interval=None)

In [12]:
completed_tasks_query = """
CREATE TABLE IF NOT EXISTS completed_tasks
(
    name varchar(255),
    tasks bigint,
    complete bigint,
    incomplete bigint
);

INSERT INTO completed_tasks
SELECT u.name as name,
    COUNT(*) as tasks,
    COUNT(NULLIF(completed, 'False') ) as complete,
    COUNT(NULLIF(completed, 'True') ) as incomplete
FROM tasks t
LEFT JOIN users u ON u.id = t.assignee_id
GROUP BY u.name;
"""

t1 = PostgresOperator(
    task_id='query_completed_tasks_v1',
    sql=completed_tasks_query,
    postgres_conn_id='asana_db', 
    autocommit=True,
    dag=dag)